In [78]:
import nltk
import numpy as np
import re
from nltk.stem import PorterStemmer

#stopwords is a list of useless words when it comes to query, words like the or is will appear several times and even tho their weight will be reduced accordingly, it is better to remove them
stopwords = ['ourselves','hers','between','yourself','but','again','there','about','once','during','out','very','having','with',
'they','own','an','be','some','for','do','its','yours','such','into','of','most','itself','other','off','is','s',
'am','or','who','as','from','him','each','the','themselves','until','below','are','we','these','your','his','through',
'don','nor','me','were','her','more','himself','this','down','should','our','their','while','above','both','up','to',
'ours','had','she','all','no','when','at','any','before','them','same','and','been','have','in','will','on','does',
'yourselves','then','that','because','what','over','why','so','can','did','not','now','under','he','you','herself','has',
'just','where','too','only','myself','which','those','i','after','few','whom','t','being','if','theirs','my','against','a',
'by','doing','it','how','further','was','here','than']

def stemming(word):
    stemmer = PorterStemmer()
    stemmed_word = stemmer.stem(word)
    return stemmed_word

def clean_doc(liste):
    # using list comprehension to perform the task, this function is removing cap letter and removing character like ? or ,
    # This version of the function doesn't use stemming
    res = []
    for word in liste :
        new_word = word.lower()
        new_word = re.sub(r'[^\w\s]', '', new_word)
        if new_word != "" and new_word not in stopwords:
            res.append(new_word)
    return res

def clean_doc_stem(liste):
    # using list comprehension to perform the task, this function is removing cap letter and removing character like ? or ,
    # This version of the function use stemming
    res = []
    for word in liste :
        new_word = word.lower()
        new_word = re.sub(r'[^\w\s]', '', new_word)
        if new_word != "" and new_word not in stopwords:
            new_word_stem = stemming(new_word)
            res.append(new_word_stem)
    return res


def indexation(document):
    # This version of the function doesn't use stemming
    words = document.split(' ')
    words = clean_doc(words)
    doc_len = len(words)
    word_count = {}
    for word in words :
        word_count[word] = 0
    for word in words :
        word_count[word] += 1
    
    return word_count, doc_len

def indexation_stem(document):
    # This version of the function use stemming
    words = document.split(' ')
    words = clean_doc_stem(words)
    doc_len = len(words)
    word_count = {}
    for word in words :
        word_count[word] = 0
    for word in words :
        word_count[word] += 1
    
    return word_count, doc_len
    

def TF(w, document):
    occurence = 0
    if w in document[1]['word_count'].keys():
        occurence = document[1]['word_count'][w]
    return occurence/document[1]['doc_len']

def iDF(numdoc, w, document):
    #division par 0 impossible
    occurence = 1
    if w in document[1]['word_count'].keys():
        occurence += 1
    return np.log(numdoc/occurence)

def tf_idf(query, table, k):
    #Prends une requete sous la forme d'une liste de mots, un jeu de document et un nombre k pour renvoyer les k premiers resultats
    tf_idf_value = {}
    numdoc = len(table.keys())
    for doc in table.items() :
        tfidf_value = 0
        for word in query :
            tf = TF(word, doc)
            idf = iDF(numdoc, word, doc)
            tfidf_value += tf*idf
        tf_idf_value[doc[0]] = tfidf_value
    tf_idf_value_list = sorted(tf_idf_value.items(), key=lambda x: x[1], reverse=True)
    return tf_idf_value_list[:k]

def loadAll(filename, stem) :
    #Fonction de parsing du fichier CISI.ALL
    word_set = []
    cisi_all = {}
    id = 0
    lines = open(filename).read()
    for document in lines.split(".I ") :
        token = ""
        buffer = ""
        cross_reference = []
        cisi_all[id] = {}
        cisi_all[id]['ID'] = id
        cisi_all[id]['titre'] = ""
        cisi_all[id]["texte"] = ""
        document += "EndOfDocument"
        for line in document.split("\n") :
            if token == ".T" and line != ".A":
                cisi_all[id]['titre'] = line.replace("\n", " ")
            elif token == ".W" and line != ".X" :
                buffer += " " + line.replace("\n", " ")
                continue
            elif line == ".X":
                cisi_all[id]["texte"] = buffer
                buffer = ""
            elif token == ".X" and line != "EndOfDocument":
                cross_reference.append(line.split("\t"))
                continue
            elif line == "EndOfDocument":
                cisi_all[id]["cross_ref"] = cross_reference
                cross_reference = []
            token = line
        if(stem == True):
            cisi_all[id]['word_count'], cisi_all[id]['doc_len'] = indexation_stem(cisi_all[id]["texte"] + " " + cisi_all[id]['titre'])
        else :
            cisi_all[id]['word_count'], cisi_all[id]['doc_len'] = indexation(cisi_all[id]["texte"] + " " + cisi_all[id]['titre'])
        for word in cisi_all[id]['word_count'].keys():
            if word not in word_set :
                word_set.append(word)
        id += 1
    del cisi_all[0]
    return cisi_all

def loadQuery(filename, stem) :
    #Fonction de parsing du fichier CISI.QRY 
    cisi_qry = {}
    id = 0
    lines = open(filename).read()
    for document in lines.split(".I ") :
        token = ""
        buffer = ""
        cisi_qry[id] = {}
        cisi_qry[id]['ID'] = id
        cisi_qry[id]["query"] = ""
        document += "EndOfDocument"
        for line in document.split("\n") :
            if token == ".W" and line != "EndOfDocument" :
                buffer += " " + line.replace("\n", " ")
                continue
            elif line == "EndOfDocument":
                cisi_qry[id]["query"] = buffer
                buffer = ""
                continue
            token = line
        if(stem == True):
            cisi_qry[id]['query'], cisi_qry[id]['doc_len'] = indexation_stem(cisi_qry[id]["query"])
        else :
            cisi_qry[id]['query'], cisi_qry[id]['doc_len'] = indexation(cisi_qry[id]["query"])
        id += 1
    del cisi_qry[0]
    return cisi_qry

def loadRel(filename) : 
    #Fonction de parsing du fichier CISI.REL 
    cisi_rel = {}
    lines = open(filename).read()
    lines += "EndOfDocument"
    buffer = 1
    list_buffer = []
    for line in lines.split("\n") :
        if(line == "EndOfDocument"):
            cisi_rel[token] = list_buffer
            return cisi_rel
        else :
            token = int((line[0:6]).replace(" ",""))
            if(token == buffer):
                list_buffer.append(int((line[7:13]).replace(" ","")))
                buffer = token
                continue
            else :
                cisi_rel[buffer] = list_buffer
                list_buffer = []
                list_buffer.append(int((line[7:13]).replace(" ","")))
                buffer = token
                continue

def tf_idf_evaluation_without_repetition(stem):
    #Calcul de la precision moyenne global du modele sans poids associe aux mots. Stem est un booleen qui definit si nous utilisons du stemming ou non
    final_list = []
    QueryDic = loadQuery("CISI.QRY", stem)
    AllDic = loadAll("CISI.ALL", stem)
    RelDic = loadRel("CISI.REL")
    for query in QueryDic.items():
        id = query[1]["ID"]
        wordlist = []
        for word in query[1]["query"].keys():
            wordlist.append(word)
        if id in RelDic.keys():
            num_pages = len(RelDic[id])
        else :
            num_pages = 10
        result = tf_idf(wordlist, AllDic, num_pages)
        if id in RelDic.keys():
            somme = 0
            for pair in result :
                if pair[0] in RelDic[id]:
                    somme += 1
            final_list.append(somme/num_pages)
    return sum(final_list)/len(final_list)

def tf_idf_evaluation_with_repetition(stem, ten):
    #Calcul de la precision moyenne global du modele avec les poids associe aux mots. Stem est un booleen qui definit si nous utilisons du stemming ou non.
    #Ten est un booleen dans le cadre du calcul de Precision@10, voir rapport.
    final_list = []
    QueryDic = loadQuery("CISI.QRY", stem)
    AllDic = loadAll("CISI.ALL", stem)
    RelDic = loadRel("CISI.REL")
    for query in QueryDic.items():
        id = query[1]["ID"]
        wordlist = []
        for word in query[1]["query"].items():
            i = 0
            while(i < word[1]):
                wordlist.append(word[0])
                i += 1
        if ten == False :
            if id in RelDic.keys():
                num_pages = len(RelDic[id])
            else :
                num_pages = 10
        else :
            num_pages = 10
        result = tf_idf(wordlist, AllDic, num_pages)
        if id in RelDic.keys():
            somme = 0
            for pair in result :
                if ten == False :
                    if pair[0] in RelDic[id]:
                        somme += 1
                else :
                    if pair[0] in RelDic[id] and len(RelDic[id]) > 9 :
                        somme += 1
            final_list.append(somme/num_pages)
    return sum(final_list)/len(final_list)



def tf_idf_given_query(queryID, stem):
    #Renvoie les 10 premiers resultats associes a un numero de requete donne
    querytest = []
    QueryDic = loadQuery("CISI.QRY", stem)
    for word in QueryDic[queryID]["query"].keys():
            querytest.append(word)    
    return tf_idf(querytest, loadAll("CISI.ALL", stem), 10)

print("\n--- TFIDF sans repetition sans stem ---\n")
print(tf_idf_evaluation_without_repetition(False))
print("\n--- TFIDF avec repetition sans stem ---\n")
print(tf_idf_evaluation_with_repetition(False, False))
print("\n--- TFIDF sans repetition avec stem ---\n")
print(tf_idf_evaluation_without_repetition(True))
print("\n--- TFIDF avec repetition avec stem ---\n")
print(tf_idf_evaluation_with_repetition(True, False))
print("\n--- TFIDF avec repetition avec stem qu'avec les 10 ---\n")
print(tf_idf_evaluation_with_repetition(True, True))
print("\n--- TFIDF query 6 avec stem ---\n")
print(tf_idf_given_query(6, True))



--- TFIDF sans repetition sans stem ---

0.11142377892985601

--- TFIDF avec repetition sans stem ---

0.12598313294016977

--- TFIDF sans repetition avec stem ---

0.12945796222453965

--- TFIDF avec repetition avec stem ---

0.14711282134750892

--- TFIDF avec repetition avec stem qu'avec les 10 ---

0.18399999999999994

--- TFIDF query 6 avec stem ---

[(1357, 0.7756522981344043), (312, 0.7063976286581182), (1077, 0.699262299075713), (326, 0.6688595904202472), (325, 0.6593044534142437), (666, 0.6593044534142437), (698, 0.6180979250758535), (1289, 0.5993676849220397), (1295, 0.5993676849220397), (398, 0.549420377845203)]


{'ID': 1, 'titre': '18 Editions of the Dewey Decimal Classifications', 'texte': "    The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.", 'cross_ref': [['1', '5', '1'], ['92', '1', '1'], ['262', '1', '1'], ['556', '1', '1'], ['1004', '1', '1'], ['1024', '1', '1'], ['1024', '1', '1']], 'word_count': {'present': 1, 'studi': 1, 'histori': 2, 'dewey': 3, 'decim': 2, 'classif': 2, 'first': 2, 'edit': 4, 'ddc': 2, 'publish': 1, '1876': 1, 'eighteenth': 1, '1971': 1, 'futur': 1, 'continu': 1, 'appear': 1, 'need': 1, 